In [14]:
# Import Basic Modules
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from glob import glob

In [15]:
# Import CNN Modules
from keras import datasets, layers, models
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model 
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [17]:
Image_size=[224,224]
train_path = 'Data/Datasets/Train'  
test_path = 'Data/Datasets/Test'

# You can add a folder inside Train and Test folders with your own pictures 
# to train the model to detect your face captured using web camera along with your name

In [4]:
vgg = VGG16(input_shape=Image_size + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False

In [19]:
folders = glob('Data/Datasets/Train/*')
folders

['Data/Datasets/Train\\Arijit',
 'Data/Datasets/Train\\emilia-clarke',
 'Data/Datasets/Train\\justin',
 'Data/Datasets/Train\\kit-harington',
 'Data/Datasets/Train\\nikolaj-coster-waldau',
 'Data/Datasets/Train\\peter-dinklage']

In [6]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
training_set = train_datagen.flow_from_directory(train_path, target_size=(224,224),
                                                 batch_size=32, class_mode ='categorical')

test_set = test_datagen.flow_from_directory(test_path, target_size=(224,224),
                                                 batch_size=32, class_mode ='categorical')

Found 38 images belonging to 6 classes.
Found 9 images belonging to 6 classes.


In [10]:
final_model = model.fit_generator(training_set, validation_data=test_set, epochs=10, steps_per_epoch=len(training_set), validation_steps=len(test_set))

C:\Users\A031828\AppData\Local\Temp/ipykernel_36588/1042045422.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final_model = model.fit_generator(training_set, validation_data=test_set, epochs=10, steps_per_epoch=len(training_set), validation_steps=len(test_set))


Epoch 1/10
2/2 [==============================] - 4s 3s/step - loss: 3.7574 - accuracy: 0.1579 - val_loss: 2.0816 - val_accuracy: 0.4444
Epoch 2/10
2/2 [==============================] - 3s 1s/step - loss: 2.7178 - accuracy: 0.4211 - val_loss: 1.6130 - val_accuracy: 0.3333
Epoch 3/10
2/2 [==============================] - 3s 1s/step - loss: 1.4642 - accuracy: 0.3947 - val_loss: 1.2901 - val_accuracy: 0.5556
Epoch 4/10
2/2 [==============================] - 4s 1s/step - loss: 0.7803 - accuracy: 0.7895 - val_loss: 1.1828 - val_accuracy: 0.4444
Epoch 5/10
2/2 [==============================] - 3s 3s/step - loss: 0.7988 - accuracy: 0.6316 - val_loss: 0.4958 - val_accuracy: 0.8889
Epoch 6/10
2/2 [==============================] - 3s 3s/step - loss: 0.5652 - accuracy: 0.7895 - val_loss: 0.4727 - val_accuracy: 0.7778
Epoch 7/10
2/2 [==============================] - 3s 3s/step - loss: 0.4750 - accuracy: 0.8684 - val_loss: 0.2991 - val_accuracy: 0.8889
Epoch 8/10
2/2 [=========================

In [11]:
from keras.models import load_model
model.save('Data/face_detection_model.h5')

# Face detection with Name

In [12]:
# Face Detection with CNN Model
from keras.models import load_model
import base64
from PIL import Image
import json
import cv2

In [13]:
model = load_model('Data/face_detection_model.h5')
face_cascade = cv2.CascadeClassifier('Data/haarcascade_frontalface_default.xml')

def face_extractor(img):
  faces=face_cascade.detectMultiScale(img, 1.5, 5)

  if faces is ():
    return None
  for (x, y, w, h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h),(0,255,255),2)
    cropped_face = img[y:y+h, x:x+w]
  return cropped_face

cap = cv2.VideoCapture(0)
while True:
  ret, frame = cap.read()
  face = face_extractor(frame)
  if type(face) is np.ndarray:
    face = cv2.resize(face,(224,224))
    im = Image.fromarray(face, 'RGB')
    img_array = np.array(im)
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)
#     print(pred)

    name = "None Matching"

  if(pred[0][2]>0.8):
    name='My Name'    # Replace with your name
    cv2.putText(frame, name, (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0),2)
  else:
    cv2.putText(frame, "No Face Found", (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0),2)
    
  cv2.imshow("image", frame)
  if cv2.waitKey(1) & 0xFF == ord('x'):
    break

cap.release()
cv2.destroyAllWindows() 

<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\A031828\AppData\Local\Temp/ipykernel_36588/2415749681.py:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


1/1 [==============================] - 0s 90ms/step


1/1 [==============================] - 0s 95ms/step
